In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train_data.csv
/kaggle/input/titanic/test_data.csv


# Import ADS

In [2]:
input_ads_pre = pd.read_csv('../input/titanic/train_data.csv')
input_ads_pre.drop(columns=['Unnamed: 0','Title_1','Title_2','Title_3','Title_4'],inplace=True) #Dropping un-necessary columns
#-----------------------------------------------------------------
print(input_ads_pre.shape)
input_ads_pre.head()

(792, 12)


,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,1,0,1,0.2750,0.014151,0,0,1,0.1,0,0,1
1,2,1,0,0.4750,0.139136,1,0,0,0.1,1,0,0
2,3,1,0,0.3250,0.015469,0,0,1,0.0,0,0,1
3,4,1,0,0.4375,0.103644,1,0,0,0.1,0,0,1
4,5,0,1,0.4375,0.015713,0,0,1,0.0,0,0,1


# Null Check

In [3]:
pd.DataFrame(input_ads_pre.isnull().sum()).T

,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,0,0,0,0,0,0,0,0,0,0,0,0


# Description of Target Variable

In [4]:
#Total survived vs not-survived split in the training data
input_ads_pre['Survived'].value_counts()

0    486
1    306
Name: Survived, dtype: int64

# Shuffling the data

In [5]:
from sklearn.utils import shuffle
#np.random.seed(100)

#----------------------------------------------------
input_ads = shuffle(input_ads_pre,random_state=100)
print(input_ads.shape)
input_ads = input_ads.reset_index(drop=True)
input_ads.head(3)

(792, 12)


,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,371,1,1,0.3125,0.108215,1,0,0,0.1,1,0,0
1,556,0,1,0.7750,0.051822,1,0,0,0.0,0,0,1
2,624,0,1,0.2625,0.015330,0,0,1,0.0,0,0,1


# Train-Test manipulation of data

In [6]:
target = 'Survived' #To predict

#--------------------------------------------------------------------------------
#Splitting into X & Y datasets (supervised training)
X = input_ads[[cols for cols in list(input_ads.columns) if target not in cols]]
y = input_ads[target]

#--------------------------------------------------------------------------------
#Since test data is already placed in the input folder separately, we will just import it
test_ads_pre = pd.read_csv('../input/titanic/test_data.csv')
test_ads_pre.drop(columns=['Unnamed: 0','Title_1','Title_2','Title_3','Title_4'],inplace=True) #Dropping un-necessary columns
test_ads = shuffle(test_ads_pre,random_state=100)
test_ads = test_ads.reset_index(drop=True)

#Splitting into X & Y datasets (supervised training)
X_test = test_ads[[cols for cols in list(test_ads.columns) if target not in cols]]
y_test = test_ads[target]

print('Train % of total data:',100 * X.shape[0]/(X.shape[0] + X_test.shape[0]))
#--------------------------------------------------------------------------------
#Manipulation of datasets for convenience and consistency
X_arr = np.array(X)
X_test_arr = np.array(X_test)

y_arr = np.array(y).reshape(X_arr.shape[0],1)
y_test_arr = np.array(y_test).reshape(X_test_arr.shape[0],1)

#--------------------------------------------------------------------------------
#Basic Summary
print(X_arr.shape)
print(X_test_arr.shape)
print(y_arr.shape)

Train % of total data: 88.78923766816143
(792, 11)
(100, 11)
(792, 1)


# Stacking Wrapper Logic from scratch

In [7]:
import warnings
warnings.filterwarnings('ignore')

# Importing all the necessary models for base model purpose

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

## UDF for Stacking Ensemble Training (Clf)

In [9]:
def stacking_ensemble_clf_training(data_x,data_y,estimator_tray,meta_estimator,passthrough=True):

    fit_level_1 = [] #Model trays for level 1
    fit_level_2 = [] #model Trays for level 2
    train_pred_arr = np.array([np.nan] * len(data_x)).reshape(len(data_x),1)

    for estimator in estimator_tray: #Creating level 1 models

        estimator.fit(data_x,data_y)
        pred_temp = np.array(estimator.predict(data_x)).reshape(len(data_x),1)
        train_pred_arr = np.append(train_pred_arr,pred_temp,axis=-1)
        fit_level_1.append(estimator)

    train_pred_arr = train_pred_arr[:,1:] #Removing the first null column

    if passthrough==False: #If training data doesnt needs to be passed to the level 2

        meta_estimator.fit(train_pred_arr,data_y)
        fit_level_2.append(meta_estimator)
        #pred_meta = meta_estimator.predict(X_test_arr)

    elif passthrough==True: #If training data needs to be passed to the level 2

        train_pass = np.append(data_x,train_pred_arr,axis=-1)
        meta_estimator.fit(train_pass,data_y)
        fit_level_2.append(meta_estimator)
        
        #pred_meta = meta_estimator.predict(X_test_arr)

    return fit_level_1,fit_level_2[0]


## UDF for Stacking Ensemble Prediction

In [10]:
def stacking_ensemble_clf_predict(data_x_test,data_y_test,fit_level_1_tray,meta_estimator,passthrough=True):

    train_pred_arr = np.array([np.nan] * len(data_x_test)).reshape(len(data_x_test),1) #Initializing level 1 prediction array

    for estimator in fit_level_1_tray: #Predcitng for each base model

        pred_temp = np.array(estimator.predict(data_x_test)).reshape(len(data_x_test),1)
        train_pred_arr = np.append(train_pred_arr,pred_temp,axis=-1)

    train_pred_arr = train_pred_arr[:,1:] #Removing the first null column

    if passthrough==False: #Should match with training

        pred_meta = meta_estimator.predict(train_pred_arr)

    elif passthrough==True: #Should match with training

        train_pass = np.append(data_x_test,train_pred_arr,axis=-1)
        pred_meta = meta_estimator.predict(train_pass)

    return pred_meta


## Invoking UDF for Stacking Ensemble Training (Clf)

In [11]:
#Level 1 models
log_reg = LogisticRegression(solver='sag',random_state=100)
sgd_clf = SGDClassifier(random_state=100)
knn = KNeighborsClassifier(n_neighbors=3)
dt_clf = DecisionTreeClassifier(random_state=100)
rf_clf = RandomForestClassifier(random_state=100)
svc = SVC(random_state=100)

xgb_ = xgb.XGBClassifier(random_state=100) #Meta Classifier
#-----------------------------------------------------------------------------------------------------------------
level_1 = [log_reg,sgd_clf,knn,dt_clf,rf_clf,svc]
#meta = xgb_

#-----------------------------------------------------------------------------------------------------------------
fit_level_1,meta_estimator = stacking_ensemble_clf_training(data_x=X_arr,
                                                            data_y=y_arr,
                                                            estimator_tray=level_1,
                                                            meta_estimator=xgb_,
                                                            passthrough=True)


[04:19:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Invoking prediction UDF for Stacking Ensemble

In [12]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

final_preds = stacking_ensemble_clf_predict(data_x_test=X_test_arr,
                                            data_y_test=y_test_arr,
                                            fit_level_1_tray=fit_level_1,
                                            meta_estimator=meta_estimator,
                                            passthrough=True)

#-------------------------------------------------------------------------------
#Evaluation of the manual ensemble
print('ROC AUC of test set :',roc_auc_score(y_test_arr,final_preds))
print('Accuracy of test set :',accuracy_score(y_test_arr,final_preds))

ROC AUC of test set : 0.7986111111111112
Accuracy of test set : 0.82


# Sklearn Benchmark

In [13]:
#Base Estimators
estimators_list = [
('log_reg', LogisticRegression(solver='sag',random_state=100)),
('sgd_clf', SGDClassifier(random_state=100)),
('knn', KNeighborsClassifier(n_neighbors=3)),
('dt_clf', DecisionTreeClassifier(random_state=100)),
('rf_clf', RandomForestClassifier(random_state=100)),
('svc', SVC(random_state=100))]

xgb_ = xgb.XGBClassifier(random_state=100) #Meta Classifier

## Sklearn implementation of Stacking

In [14]:
from sklearn.ensemble import StackingClassifier

stacking = StackingClassifier(estimators=estimators_list,
                              final_estimator=xgb_,
                              stack_method='predict',
                              passthrough=True,
                              n_jobs=-1)

stacking.fit(X_arr,y_arr)
stacking_pred = stacking.predict(X_test_arr)

#--------------------------------------------------------------------------
#Evaluating
print('ROC AUC of test set :',roc_auc_score(y_test_arr,stacking_pred))
print('Accuracy of test set :',accuracy_score(y_test_arr,stacking_pred))

[04:20:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC of test set : 0.7864583333333333
Accuracy of test set : 0.82


### Insights: The accuracy and ROC-AUC scores for both sklearn and manual implementations are same indicating correct implementation of the logic

# END